# Amazon product finder using selenium

En este proyecto veremos cómo se pueden obtener la descripción, los precios, las imágenes, la valoreción, etc, de cualquier producto de Amazon. 

Esto nos puede servir, por ejemplo para ver cómo están los productos con respecto a la competencia (que en este caso es Amazon)

Pasos que vamos a dar:
- Haremos una búsqueda de gafas de sol en la página web de Amazon
- Leeremos el título de cada gafa de sol que aparece debajo de su fotografía.
- Sacaremos los precios de cada una.
- Obtendremos la foto de cada una.
- Finalmente lo pondremos todo en un dataframe y lo guardaremos en un archivo csv y en un html.

NOTA ... Hay que tener en cuenta que no todos tienen precio disponible por lo que se puede romper el proceso.

### **Uso de Selenium**

En caso de que con requests nos de un código 500, 503, etc
Selenium lo que hace es poner los user_id para que Amazon no detecte que eres un boot.
Selenium no usa requests, el propio selenium se conecta a la página web y me permite hacer clicks --> es mucho más útil

- Para usar selenium es mejor hacerlo en local y no en cloud: en colab da problemas, mejor usarlo en Jupyter Notebook o Visual Studio Code
- Selenium necesita un webdriver (navegador) --> en nuestro caso usamos en chrome 
- Si lo ejecutamos en Visual Studio Code, se abre una ventana nueva, que será de chrome si usamos su webdriver y todo lo que metamos en selenium aparecerá en el chrome

**Instalación de dependencias**

In [ ]:
#!pip install selenium
#!pip install --upgrade selenium
#!pip3 install beautifulsoup4 --user
#!pip3 install lxml --user
#!pip3 install html5lib --user
#!pip3 install image
#!pip3 install webdriver_manager

**Importación de dependencias**

In [23]:
import os
from os.path import basename
#import time
import pandas as pd
#import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [7]:
# Crear un objeto de ChromeOptions
chrome_options = Options()

# Evitar que se muestre la pantalla de bienvenida
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--no-default-browser-check")
# Establezco un perfil que tengo definido para evitar que salte la ventana de elegir el perfil
chrome_options.add_argument("user-data-dir=C:/Users/jagui/AppData/Local/Google/Chrome/User Data/Profile 1")
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-infobars")
#chrome_options.add_argument("--disable-notifications")
#chrome_options.add_argument("--disable-popup-blocking")
#chrome_options.add_argument("--disable-default-apps")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox")
#chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_argument("--remote-debugging-port=9222")

# Si quieres usar un perfil de usuario existente
# Ruta al perfil de usuario de Chrome
# chrome_options.add_argument("user-data-dir=/path/to/your/chrome/profile")

# Configurar el servicio con ChromeDriverManager
service = Service(ChromeDriverManager().install())

# Crear una nueva instancia del WebDriver de Chrome con las opciones
driver = webdriver.Chrome(service=service, options=chrome_options)

**Localización de elementos**

Para localizar elementos en la página podemos usar la clase By con los siguientes atributos:
- ID = "id"
- NAME = "name"
- XPATH = "xpath"
- LINK_TEXT = "link text"
- PARTIAL_LINK_TEXT = "partial link text"
- TAG_NAME = "tag name"
- CLASS_NAME = "class name"
- CSS_SELECTOR = "css selector"

Mas información: https://selenium-python.readthedocs.io/locating-elements.html

*En caso de tener problemas con el bot abriremos el navegador usando el siguiente código*

In [ ]:
url = "https://www.amazon.es/"
driver.get(url)

# Hacemos una búsqueda en el search box pasar la prueba antiboot
elem = driver.find_element(By.NAME, 'field-keywords')  # Encuentra el buscador
elem.send_keys('YYCCAN' + Keys.RETURN) # Introducimos la palabra que nos indican en el navegador

# Aceptación de cookies
try:
    # Aceptamos las cookies de la página (necesario la primera vez)
    acept = driver.find_element(By.NAME, 'agree')
    acept.click()
    print("Las cookies han sido aceptadas")
    
except:
    pass
    
print("Las cookies ya se han aceptado con anterioridad")

**Búsqueda de gafas de sol**

In [8]:
# Genero una palabra clave para cada vez que quiera buscar algo, sólo tenga que cambiar la palabra
keyword="gafas de sol"

# Hago esto en vez del requests y pongo keyword en vez de iphone ... aquí no se abre ninguna ventana ... en el driver ya está la informacion de amazon
# Cargamos la primera página de resultados
driver.get("https://www.amazon.es/s?k="+keyword)

# Aceptación de cookies
try:
    # Aceptamos las cookies de la página (necesario la primera vez)
    acept = driver.find_element(By.NAME, 'agree')
    acept.click()
    print("Las cookies han sido aceptadas")
    
except:
    pass
    
print("Las cookies ya se han aceptado con anterioridad")

Las cookies ya se han aceptado con anterioridad


In [11]:
# Para ver el text .... es el equivalente a requests.get("https://www.amazon.es/").text
content = driver.page_source

# Analizamos el contenido HTML usando BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Imprimimos el contenido web
print(soup)

<html class="a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-hires a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition a-ember" data-19ax5a9jf="dingo" data-aui-build-date="3.24.6-2024-08-16" lang="es-es"><!-- sp:feature:head-start --><head><script async="" crossorigin="anonymous" src="https://images-eu.ssl-images-amazon.com/images/I/31bJewCvY-L.js"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetc

Vamos a tener diferente número de productos y de precios. Por lo tanto, hay modelos que no tienen precio, por lo que al hacer dos listas diferentes y luego querer juntarlas, tendremos problemas.

Tenemos que hacer un for loop por cada bloque de gafas (foto, título y precio) --> tenemos que buscar la clase correspondiente.

In [12]:
# Cojemos la clase de todo el bloque
resultado = soup.find_all(class_ ="a-section a-spacing-base")
len(resultado)

En "resultado" tengo toda la información y la tengo en html. 

Si el resultado me da cero, es porque la estructura de otro producto ha cambiado dentro de amazon. Aquí ponemos la class del nuevo producto.

In [14]:
if len(resultado) == 0:
    resultado = soup.find_all(class_ ="a-section a-spacing-base a-text-center")

len(resultado)

In [16]:
# Toda la información del primer producto
resultado[0]

<div class="a-section a-spacing-base a-text-center"><div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-vrkzrcf5p7mia2ppuqe05j3jfq" data-cy="image-container" style="padding-top: 0px !important;"><div class="s-image-padding"><span class="rush-component" data-component-type="s-product-image" data-render-id="ruhx5yro3i22923ixgbkcddke6" data-version-id="vrkzrcf5p7mia2ppuqe05j3jfq"><a class="a-link-normal s-no-outline" href="/sspa/click?ie=UTF8&amp;spc=MToxOTMxOTE3NzczNTMyMzgxOjE3MjQyNTQ2OTI6c3BfYXRmOjMwMDE4MDM1NTI5OTMzMjo6MDo6&amp;url=%2FLINVO-Polarizadas-Conducci%25C3%25B3n-Protecci%25C3%25B3n-Cuadradas%2Fdp%2FB0CF839JNB%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.6UpBsfoNuk3cV989_OMAoK9udCzKt4pjg877lg7DBY87ryRZ-zDxTG3mLrNiZWw1HpTlEEG3iVE4V6KZs92qOM1HLThSNhhaYbsVZknGx6lyRD0vCKf7qIOHRicK1xFenDYpltoQOADn0rz4AC3fmW4o9rA-vx_2uBNXyHsodYO2SgXJatQ2pH7W

**Guardamos la información (nombre, precio y foto) de los productos**

In [18]:
# Generamos una carpeta para guardar las imágenes
if not os.path.exists("./Imagenes_"+keyword):
    os.makedirs("./Imagenes_"+keyword)

PASOS QUE VAMOS A SEGUIR:

- En resultado tengo toda la información y la tengo en html. Entonces con un for loop la voy a recorrer haciendo las diferentes consultas.
- Esas consultas las guardamos en una lista.
- Finalmente, creamos un dataframe con los elementos de la lista.
- Guardamos el dataframe en un archivo csv y en un html.

In [21]:
# Directorio para guardar las imágenes
carpeta = "./Imagenes_"+keyword+"/"

# Lista para almacenar los datos antes de convertirlo a DataFrame
data = []

for elemento in resultado:
    titulo_producto = elemento.find(class_= "a-size-base-plus a-color-base a-text-normal")
    # Verificamos si el título del producto existe
    titulo_producto = titulo_producto.get_text() if titulo_producto else "Título no disponible"
    
    precio_producto = elemento.find(class_= "a-price-whole")
    # Verificamos si el precio del producto existe    
    precio_producto = precio_producto.get_text() if precio_producto else "No existe precio"
    
    imagen_producto = elemento.find("img") 
    # Dentro de "img" hay una serie de atributos que quiero obtener
    # Quiero obtener el link de la imagen --> src
    
    # Verificamos si la imagen del producto exite
    if imagen_producto and 'src' in imagen_producto.attrs:
        imagen_url = imagen_producto.attrs["src"]
        imagen_nombre = basename(imagen_url)
        
        # Descargo y guardo la imagen
        try:
            with open(carpeta + imagen_nombre, "wb") as fichero:
                # Utilizo el requests para descargar el contenido
                fichero.write(requests.get(imagen_url).content)
        except Exception as e:
            print(f"Error al descargar la imagen {imagen_url}: {e}")   
        
        # Añadimos los datos a la lista
        data.append({
            "titulo": titulo_producto,
            "precio": precio_producto,
            "imagen": f"<img src='{imagen_url}'>" # lo pongo en codigo HTML para que las imágenes no se vean como texto plano
        })   

In [24]:
# Generamos un DataFrame con los datos guardados en la lista
lista_productos_df = pd.DataFrame(data)
lista_productos_df

,titulo,precio,imagen
0,"Gafas de Sol Polarizadas Hombre Mujer, Gafas d...","24,",<img src='https://m.media-amazon.com/images/I/...
1,Gafas de Sol Polarizadas para Hombre Mujer Pil...,"19,",<img src='https://m.media-amazon.com/images/I/...
2,Gafas de Sol Polarizadas Hombre Mujer Moda Som...,"24,",<img src='https://m.media-amazon.com/images/I/...
3,Gafas de Sol Hombre Mujer Polarizadas Deportiv...,"24,",<img src='https://m.media-amazon.com/images/I/...
4,"3 Pares Gafas de Sol Polarizadas, Gafas Polari...","11,",<img src='https://m.media-amazon.com/images/I/...
...,...,...,...
60,Northweek Regular Sunglasses,"13,",<img src='https://m.media-amazon.com/images/I/...
61,"5039/S 9O 807 Gafas de Sol, Negro (Black/Dark ...","49,",<img src='https://m.media-amazon.com/images/I/...
62,Hombre Gafas de sol Polarizado Al-Mg Metal Sup...,"28,",<img src='https://m.media-amazon.com/images/I/...
63,Gradiant Gafas de Sol Unisex Adulto,"17,",<img src='https://m.media-amazon.com/images/I/...


In [25]:
# Filtramos los productos sin título y sin precio
lista_productos_df_final = lista_productos_df[~((lista_productos_df['titulo'] == 'Título no disponible') & (lista_productos_df['precio'] == 'No existe precio'))]
lista_productos_df_final

,titulo,precio,imagen
0,"Gafas de Sol Polarizadas Hombre Mujer, Gafas d...","24,",<img src='https://m.media-amazon.com/images/I/...
1,Gafas de Sol Polarizadas para Hombre Mujer Pil...,"19,",<img src='https://m.media-amazon.com/images/I/...
2,Gafas de Sol Polarizadas Hombre Mujer Moda Som...,"24,",<img src='https://m.media-amazon.com/images/I/...
3,Gafas de Sol Hombre Mujer Polarizadas Deportiv...,"24,",<img src='https://m.media-amazon.com/images/I/...
4,"3 Pares Gafas de Sol Polarizadas, Gafas Polari...","11,",<img src='https://m.media-amazon.com/images/I/...
...,...,...,...
60,Northweek Regular Sunglasses,"13,",<img src='https://m.media-amazon.com/images/I/...
61,"5039/S 9O 807 Gafas de Sol, Negro (Black/Dark ...","49,",<img src='https://m.media-amazon.com/images/I/...
62,Hombre Gafas de sol Polarizado Al-Mg Metal Sup...,"28,",<img src='https://m.media-amazon.com/images/I/...
63,Gradiant Gafas de Sol Unisex Adulto,"17,",<img src='https://m.media-amazon.com/images/I/...


In [26]:
# Lo guardamos en un csv
lista_productos_df_final.to_csv(keyword + ".csv")

# Lo guardamos como un HTML
lista_productos_df_final.to_html(keyword + ".html", escape=False) # escape=False es para que se vea la imagen

In [27]:
# Cerramos el navegador
driver.quit()